Лабораторная работа - тренировка ЛЛМ на датасете [DocVQA](https://rrc.cvc.uab.es/?ch=17&com=introduction). <br>
Датасет содержит анотации с вопросами к документам и OCR по словам.

Задачами лабораторной будут:

1. Тренировка phi-2 
2. Тренировка tinyLlama

Промпт для моделей будет представлять из себя